# Notes

- it seems that the TF-DF is going to the right node if the condition is met
    - if value > 3 go to right node
    - vs the others trees .... if value < 3 go to left
        - It's the same thing, but ask from different angle
    

In [1]:
%load_ext autoreload
%autoreload 2

# Imports

In [2]:
import sys
import os
# add library module to PYTHONPATH
sys.path.append(f"{os.getcwd()}/../")

from dtreeviz.models.tfdf_decision_tree import ShadowTFDFTree

import tensorflow_decision_forests as tfdf
import tensorflow_decision_forests
import pandas as pd

from sklearn.model_selection import train_test_split
from collections import defaultdict

# Data preprocessing

In [22]:
random_state = 1234
dataset = pd.read_csv("../data/titanic/titanic.csv")
# Fill missing values for Age
dataset.fillna({"Age":dataset.Age.mean()}, inplace=True)
# Encode categorical variables
dataset["Sex_label"] = dataset.Sex.astype("category").cat.codes
dataset["Cabin_label"] = dataset.Cabin.astype("category").cat.codes
dataset["Embarked_label"] = dataset.Embarked.astype("category").cat.codes
features = ["Pclass", "Age", "Fare", "Sex_label", "Cabin_label", "Embarked_label"]
target = "Survived"

In [24]:
dataset[features + [target]]

,Pclass,Age,Fare,Sex_label,Cabin_label,Embarked_label,Survived
0,3,22.000000,7.2500,1,-1,2,0
1,1,38.000000,71.2833,0,81,0,1
2,3,26.000000,7.9250,0,-1,2,1
3,1,35.000000,53.1000,0,55,2,1
4,3,35.000000,8.0500,1,-1,2,0
...,...,...,...,...,...,...,...
886,2,27.000000,13.0000,1,-1,2,0
887,1,19.000000,30.0000,0,30,2,1
888,3,29.699118,23.4500,0,-1,2,0
889,1,26.000000,30.0000,1,60,0,1


# Classification

## Model training

In [28]:
train, test = train_test_split(dataset[features + [target]], train_size=0.7, stratify=dataset.Survived)

In [29]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train, label=target)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test, label=target)

In [30]:
model = tfdf.keras.RandomForestModel(max_depth=3)


Use /var/folders/yp/rczwnkhn6nn9mfcf0bgc61jh0000gn/T/tmp1zpss3y5 as temporary training directory


In [31]:
model.fit(train_ds)


Starting reading the dataset
1/1 [==============================] - ETA: 0s
Dataset read in 0:00:00.127191
Training model
Model trained in 0:00:00.022056
Compiling model
1/1 [==============================] - 0s 159ms/step


In [32]:
model.compile(metrics=["accuracy"])


In [33]:
print(model.evaluate(test_ds))


1/1 [==============================] - 0s 321ms/step - loss: 0.0000e+00 - accuracy: 0.7649
[0.0, 0.7649253606796265]


## Model metadata discovery

### Prepare for HTML

In [306]:

print(tfdf.model_plotter.plot_model(model, tree_idx=1))


<script src="https://d3js.org/d3.v6.min.js"></script>
<div id="tree_plot_0a120c0fa7d04e6d8495fd9bff8c09f6"></div>
<script>
/*
 * Copyright 2021 Google LLC.
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 *     https://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

/**
 *  Plotting of decision trees generated by TF-DF.
 *
 *  A tree is a recursive structure of node objects.
 *  A node contains one or more of the following components:
 *
 *    - A value: Representing the output of the node. If the node is not a leaf,
 *      the value is only 

### Model inspector

In [50]:
model.make_inspector()

In [308]:
dir(model.make_inspector())

['MODEL_NAME',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_dataspec',
 '_directory',
 '_header',
 '_make_simple_column_spec',
 '_specialized_header',
 'dataspec',
 'evaluation',
 'export_to_tensorboard',
 'extract_all_trees',
 'extract_tree',
 'features',
 'header',
 'iterate_on_nodes',
 'label',
 'label_classes',
 'metadata',
 'model_type',
 'num_trees',
 'objective',
 'specialized_header',
 'task',
 'training_logs',
 'variable_importances',
 'winner_take_all_inference']

In [309]:
for node in model.make_inspector().iterate_on_nodes():
    if node.tree_idx == 1:
        print(node)
        print(node.tree_idx)
        print("----")

IterNodeResult(node=NonLeafNode(condition=(Sex_label in [1]; miss=False), pos_child=None, neg_child=None, value=ProbabilityValue([0.608346709470305, 0.391653290529695],n=623.0)), depth=0, tree_idx=1)
1
----
IterNodeResult(node=NonLeafNode(condition=(Fare >= 26.268749237060547; miss=True), pos_child=None, neg_child=None, value=ProbabilityValue([0.7990314769975787, 0.2009685230024213],n=413.0)), depth=1, tree_idx=1)
1
----
IterNodeResult(node=LeafNode(value=ProbabilityValue([0.8720538720538721, 0.12794612794612795],n=297.0), idx=0), depth=2, tree_idx=1)
1
----
IterNodeResult(node=LeafNode(value=ProbabilityValue([0.6120689655172413, 0.3879310344827586],n=116.0), idx=1), depth=2, tree_idx=1)
1
----
IterNodeResult(node=NonLeafNode(condition=(Pclass >= 2.5; miss=False), pos_child=None, neg_child=None, value=ProbabilityValue([0.23333333333333334, 0.7666666666666667],n=210.0)), depth=1, tree_idx=1)
1
----
IterNodeResult(node=LeafNode(value=ProbabilityValue([0.03225806451612903, 0.9677419354838

In [25]:
tree = model.make_inspector().extract_tree(tree_idx=1)

In [26]:
print(tree)

Tree(NonLeafNode(condition=(Sex_label in [1]; miss=False), pos_child=NonLeafNode(condition=(Pclass >= 2.5; miss=False), pos_child=LeafNode(value=ProbabilityValue([0.5402298850574713, 0.45977011494252873],n=87.0), idx=3), neg_child=LeafNode(value=ProbabilityValue([0.06299212598425197, 0.937007874015748],n=127.0), idx=2), value=ProbabilityValue([0.2570093457943925, 0.7429906542056075],n=214.0)), neg_child=NonLeafNode(condition=(Age >= 13.0; miss=True), pos_child=LeafNode(value=ProbabilityValue([0.8601583113456465, 0.13984168865435356],n=379.0), idx=1), neg_child=LeafNode(value=ProbabilityValue([0.3, 0.7],n=30.0), idx=0), value=ProbabilityValue([0.8190709046454768, 0.18092909535452323],n=409.0)), value=ProbabilityValue([0.6260032102728732, 0.3739967897271268],n=623.0)),label_classes={self.label_classes})


In [27]:
def get_nodes_info(root_node):
    tree_nodes = defaultdict(lambda: None)
    children_left = defaultdict(lambda: -1)
    children_right = defaultdict(lambda: -1)
    node_index = 0
    
    def recur(node, node_id):
        nonlocal node_index
        
        tree_nodes[node_id] = node
        
        if isinstance(node, tensorflow_decision_forests.component.py_tree.node.LeafNode):
            return
        else:
            node_index += 1
            children_left[node_id] = node_index
            recur(node.neg_child, node_index)
            
            node_index += 1
            children_right[node_id] = node_index
            recur(node.pos_child, node_index)
    
    recur(root_node, 0)
    
    return tree_nodes, children_left, children_right
    

In [314]:
tree_nodes, children_left, children_right = get_nodes_info(tree.root)

In [315]:
children_left

defaultdict(<function __main__.get_nodes_info.<locals>.<lambda>()>,
            {0: 1, 1: 2, 4: 5})

In [316]:
children_right

defaultdict(<function __main__.get_nodes_info.<locals>.<lambda>()>,
            {1: 3, 0: 4, 4: 6})

In [317]:
tree_nodes

defaultdict(<function __main__.get_nodes_info.<locals>.<lambda>()>,
            {0: NonLeafNode(condition=(Sex_label in [1]; miss=False), pos_child=NonLeafNode(condition=(Pclass >= 2.5; miss=False), pos_child=LeafNode(value=ProbabilityValue([0.5232558139534884, 0.47674418604651164],n=86.0), idx=3), neg_child=LeafNode(value=ProbabilityValue([0.03225806451612903, 0.967741935483871],n=124.0), idx=2), value=ProbabilityValue([0.23333333333333334, 0.7666666666666667],n=210.0)), neg_child=NonLeafNode(condition=(Fare >= 26.268749237060547; miss=True), pos_child=LeafNode(value=ProbabilityValue([0.6120689655172413, 0.3879310344827586],n=116.0), idx=1), neg_child=LeafNode(value=ProbabilityValue([0.8720538720538721, 0.12794612794612795],n=297.0), idx=0), value=ProbabilityValue([0.7990314769975787, 0.2009685230024213],n=413.0)), value=ProbabilityValue([0.608346709470305, 0.391653290529695],n=623.0)),
             1: NonLeafNode(condition=(Fare >= 26.268749237060547; miss=True), pos_child=LeafNode(v

### Save the model

In [59]:
from tensorflow.keras.models import Model

In [57]:
model.save("/Users/tudor.lapusan/Documents/workspace/personal/dtreeviz/testing/testlib/models/fixtures/tfdf_classifier")


INFO:tensorflow:Assets written to: /Users/tudor.lapusan/Documents/workspace/personal/dtreeviz/testing/testlib/models/fixtures/tfdf_classifier/assets


INFO:tensorflow:Assets written to: /Users/tudor.lapusan/Documents/workspace/personal/dtreeviz/testing/testlib/models/fixtures/tfdf_classifier/assets


In [18]:
import tensorflow

In [19]:
model2 = tensorflow.keras.models.load_model("/Users/tudor.lapusan/Documents/workspace/personal/dtreeviz/testing/testlib/models/fixtures/tfdf_classifier_0.2.4")


In [34]:
model2.predict(test_ds)

ValueError: in user code:

    File "/Users/tudor.lapusan/Documents/workspace/personal/dtreeviz/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "/Users/tudor.lapusan/Documents/workspace/personal/dtreeviz/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/tudor.lapusan/Documents/workspace/personal/dtreeviz/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/tudor.lapusan/Documents/workspace/personal/dtreeviz/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1751, in predict_step
        return self(x, training=False)
    File "/Users/tudor.lapusan/Documents/workspace/personal/dtreeviz/venv/lib/python3.8/site-packages/keras/utils/traceback_utils.py", lin

In [66]:
model2

SyntaxError: invalid syntax (<ipython-input-66-4ee7f90b2e24>, line 1)

In [51]:
type(model)

tensorflow_decision_forests.keras.RandomForestModel

In [55]:
type(model2)

keras.saving.saved_model.load.RandomForestModel

In [69]:
from tensorflow_decision_forests.component.inspector import inspector as inspector_lib


In [80]:
insp = inspector_lib.make_inspector("/Users/tudor.lapusan/Documents/workspace/personal/dtreeviz/testing/testlib/models/fixtures/tfdf_classifier_0.2.4/assets")


In [78]:
insp

In [76]:
insp.extract_tree(tree_idx=1)

Tree(NonLeafNode(condition=(Sex_label in [1]; miss=False), pos_child=NonLeafNode(condition=(Pclass >= 2.5; miss=False), pos_child=LeafNode(value=ProbabilityValue([0.6, 0.4],n=105.0), idx=3), neg_child=LeafNode(value=ProbabilityValue([0.06837606837606838, 0.9316239316239316],n=117.0), idx=2), value=ProbabilityValue([0.31981981981981983, 0.6801801801801802],n=222.0)), neg_child=NonLeafNode(condition=(Age >= 9.5; miss=True), pos_child=LeafNode(value=ProbabilityValue([0.8337801608579088, 0.16621983914209115],n=373.0), idx=1), neg_child=LeafNode(value=ProbabilityValue([0.4642857142857143, 0.5357142857142857],n=28.0), idx=0), value=ProbabilityValue([0.8079800498753117, 0.19201995012468828],n=401.0)), value=ProbabilityValue([0.6340288924558587, 0.36597110754414125],n=623.0)),label_classes={self.label_classes})

In [77]:
tensorflow_decision_forests.__version__

'0.2.4'

## dtreeviz - model interpretation

In [10]:
tfdf_shadow = ShadowTFDFTree(model, tree_index=1, 
                             x_data=train[features], 
                             y_data=train[target], 
                             feature_names=features, 
                             target_name=target,
                             class_names=[0, 1])

In [11]:
tfdf_shadow.children_left

defaultdict(<function dtreeviz.models.tfdf_decision_tree.ShadowTFDFTree._get_nodes_info.<locals>.<lambda>()>,
            {0: 1, 1: 2, 4: 5, 2: -1, 3: -1, 5: -1, 6: -1})

# Unit tests
This section was using during development phase and unit testing

In [12]:
from tensorflow_decision_forests.component.inspector import inspector as inspector_lib


In [40]:
random_state = 1234
dataset = pd.read_csv("../testing/testlib/models/fixtures/dataset_spark_tf.csv")
features_clf = ["Pclass","Sex_label","Embarked_label", "Age_mean","SibSp","Parch","Fare"]
target_clf = "Survived"

In [4]:
train_clf = tfdf.keras.pd_dataframe_to_tf_dataset(dataset, label=target_clf)
model_clf = tfdf.keras.RandomForestModel(max_depth=3, random_seed = random_state)
model_clf.fit(train_clf)

In [7]:
print(tfdf.model_plotter.plot_model(model_clf, tree_idx=0))


<script src="https://d3js.org/d3.v6.min.js"></script>
<div id="tree_plot_a56ec324d1604f218f105dc50d6aa3ca"></div>
<script>
/*
 * Copyright 2021 Google LLC.
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 *     https://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

/**
 *  Plotting of decision trees generated by TF-DF.
 *
 *  A tree is a recursive structure of node objects.
 *  A node contains one or more of the following components:
 *
 *    - A value: Representing the output of the node. If the node is not a leaf,
 *      the value is only 

In [9]:
# model_clf.save("/Users/tudor.lapusan/Documents/workspace/personal/dtreeviz/testing/testlib/models/fixtures/tfdf_classifier_0.2.4")


INFO:tensorflow:Assets written to: /Users/tudor.lapusan/Documents/workspace/personal/dtreeviz/testing/testlib/models/fixtures/tfdf_classifier_0.2.4/assets


INFO:tensorflow:Assets written to: /Users/tudor.lapusan/Documents/workspace/personal/dtreeviz/testing/testlib/models/fixtures/tfdf_classifier_0.2.4/assets


In [13]:
insp = inspector_lib.make_inspector("/Users/tudor.lapusan/Documents/workspace/personal/dtreeviz/testing/testlib/models/fixtures/tfdf_classifier_0.2.4/assets")


In [15]:
insp

In [16]:
isinstance(insp, tensorflow_decision_forests.component.inspector.inspector._RandomForestInspector)

True

In [41]:
insp.extract_tree(tree_idx=0).root

NonLeafNode(condition=(Sex_label >= 0.5; miss=False), pos_child=NonLeafNode(condition=(Parch >= 2.5; miss=False), pos_child=LeafNode(value=ProbabilityValue([0.875, 0.125],n=16.0), idx=3), neg_child=LeafNode(value=ProbabilityValue([0.18691588785046728, 0.8130841121495327],n=321.0), idx=2), value=ProbabilityValue([0.2195845697329377, 0.7804154302670623],n=337.0)), neg_child=NonLeafNode(condition=(Pclass >= 1.5; miss=True), pos_child=LeafNode(value=ProbabilityValue([0.851528384279476, 0.14847161572052403],n=458.0), idx=1), neg_child=LeafNode(value=ProbabilityValue([0.5729166666666666, 0.4270833333333333],n=96.0), idx=0), value=ProbabilityValue([0.8032490974729242, 0.1967509025270758],n=554.0)), value=ProbabilityValue([0.5824915824915825, 0.4175084175084175],n=891.0))

In [35]:
model2 = tensorflow.keras.models.load_model("/Users/tudor.lapusan/Documents/workspace/personal/dtreeviz/testing/testlib/models/fixtures/tfdf_classifier_0.2.4")


In [ ]:
model2.